In [ ]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import os
import pandas as pd
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


In [ ]:

# Step 1: Preprocess the image
def preprocess_image(image_path, target_size=(224, 224)):
    """Resize and normalize the image."""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at {image_path} could not be loaded.")
    resized_image = cv2.resize(image, target_size)
    normalized_image = resized_image.astype(np.float32) / 255.0
    return normalized_image


In [ ]:
# Step 2: Extract features using a pre-trained CNN
def extract_features_with_cnn(image):
    """Extract high-level features using MobileNetV2."""
    base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
    


    
    # Extract features
    features = base_model.predict(np.expand_dims(image, axis=0))
    return features.flatten()  # Flatten to a 1D vector


In [ ]:

# Step 3: Cluster features using K-Means
def cluster_features(features, k=5):
    """Cluster features using K-Means."""
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features.reshape(-1, 1))  # Reshape for clustering
    dominant_colors = kmeans.cluster_centers_
    labels = kmeans.labels_
    return labels, dominant_colors



In [ ]:
# Step 4: Reconstruct the image
def reconstruct_image(labels, dominant_colors, image_shape):
    """Reconstruct the image using dominant colors."""
    reconstructed = dominant_colors[labels].reshape(image_shape[0], image_shape[1], 3)
    reconstructed = (reconstructed * 255).astype(np.uint8)
    return reconstructed

# Step 5: Calculate metrics
def calculate_mse(original_image, reconstructed_image):
    """Calculate Mean Squared Error (MSE)."""
    if original_image.shape != reconstructed_image.shape:
        reconstructed_image = cv2.resize(reconstructed_image, (original_image.shape[1], original_image.shape[0]))
    return np.mean((original_image.astype(float) - reconstructed_image.astype(float)) ** 2)

def calculate_ssim(original_image, reconstructed_image):
    """Calculate Structural Similarity Index (SSIM)."""
    if original_image.shape != reconstructed_image.shape:
        reconstructed_image = cv2.resize(reconstructed_image, (original_image.shape[1], original_image.shape[0]))
    if len(original_image.shape) == 3:
        original_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        reconstructed_gray = cv2.cvtColor(reconstructed_image, cv2.COLOR_BGR2GRAY)
    else:
        original_gray = original_image
        reconstructed_gray = reconstructed_image
    return ssim(original_gray, reconstructed_gray, data_range=reconstructed_gray.max() - reconstructed_gray.min())



In [ ]:
# Step 6: Visualize results
def visualize_results(original_image, reconstructed_image, filename):
    """Visualize original and reconstructed images."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")
    axes[0].axis("off")
    
    axes[1].imshow(cv2.cvtColor(reconstructed_image, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Reconstructed Image")
    axes[1].axis("off")
    
    plt.tight_layout()
    plt.savefig(f"results_{filename}.png")
    plt.show()



In [ ]:
# Step 7: Process all images in a folder
def process_images_in_folder(folder_path, k=5, output_csv="metrics_cnn_k_5.csv"):
    """Process all images in a folder, compute metrics, and save results."""
    if not os.path.isdir(folder_path):
        raise ValueError(f"Directory {folder_path} does not exist.")
    
    metrics = []
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)
    
    print(f"\nFound {total_images} images. Processing...\n")
    
    for idx, filename in enumerate(sorted(image_files), 1):
        image_path = os.path.join(folder_path, filename)
        try:
            print(f"Processing image {idx}/{total_images}: {filename}")
            
            # Preprocess the image
            preprocessed_image = preprocess_image(image_path)
            
            # Extract features using CNN
            features = extract_features_with_cnn(preprocessed_image)
            
            # Cluster features
            labels, dominant_colors = cluster_features(features, k=k)
            
            # Reconstruct the image
            reconstructed_image = reconstruct_image(labels, dominant_colors, preprocessed_image.shape[:2])
            
            # Load original image for metrics
            original_image = cv2.imread(image_path)
            
            # Calculate metrics
            mse = calculate_mse(original_image, reconstructed_image)
            ssim_value = calculate_ssim(original_image, reconstructed_image)
            
            # Append metrics
            metrics.append({
                "Image": filename,
                "k": k,
                "MSE": mse,
                "SSIM": ssim_value,
                "Dominant Colors": str(dominant_colors.tolist())
            })
            
            # Visualize results
            visualize_results(original_image, reconstructed_image, filename)
        
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    
    # Save metrics to CSV
    df = pd.DataFrame(metrics)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Processed {len(metrics)}/{total_images} images successfully.")
    print(f"Metrics saved to {output_csv}")
    return df



: 

In [ ]:
# Main execution
if __name__ == "__main__":
    image_folder = "/home/sala/data/organized_data/preprocessed_train"
    batch_size = 1000  # Number of images to process at a time
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    total_images = len(image_files)
    print(f"Total images found: {total_images}")
    
    for i in range(0, total_images, batch_size):
        batch_files = image_files[i:i + batch_size]
        print(f"\nProcessing batch {i // batch_size + 1}: {len(batch_files)} images")
        
        # Process the current batch
        batch_folder = os.path.join(image_folder, f"batch_{i // batch_size + 1}")
        os.makedirs(batch_folder, exist_ok=True)
        
        for file in batch_files:
            src_path = os.path.join(image_folder, file)
            dst_path = os.path.join(batch_folder, file)
            os.symlink(src_path, dst_path)  # Create symbolic links for batch processing
        
        process_images_in_folder(batch_folder, k=5, output_csv=f"color_metrics_cnn_batch_{i // batch_size + 1}.csv")


Found 1000 images. Processing...

Processing image 1/1000: 10BG_10.0_1.png


/tmp/ipykernel_68234/3164171630.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
I0000 00:00:1743253785.607058   68234 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1743253788.376589   68483 service.cc:152] XLA service 0x7f19f803b6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743253788.376680   68483 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-03-29 18:39:48.420981: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRO

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


I0000 00:00:1743253792.043233   68483 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
/tmp/ipykernel_68234/3164171630.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


Error processing 10BG_10.0_1.png: cannot reshape array of size 1280 into shape (224,224,3)
Processing image 2/1000: 10BG_10.0_3.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Error processing 10BG_10.0_3.png: cannot reshape array of size 1280 into shape (224,224,3)
Processing image 3/1000: 10BG_2.0_2.png


...

/tmp/ipykernel_68234/3164171630.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 44s 44s/step
Error processing 2.5R_8.0_8.png: cannot reshape array of size 1280 into shape (224,224,3)
Processing image 321/1000: 2.5R_8.5_3.png


/tmp/ipykernel_68234/3164171630.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
2025-03-29 19:00:20.182315: E external/local_xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module a_inference_one_step_on_data_1233078__.1149] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2025-03-29 19:00:38.867775: E external/local_xla/xla/service/slow_operation_alarm.cc:140] The operation took 2m18.690188654s

********************************
[Compiling module a_inference_one_step_on_data_1233078__.1149] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


1/1 ━━━━━━━━━━━━━━━━━━━━ 235s 235s/step
Error processing 2.5R_8.5_3.png: cannot reshape array of size 1280 into shape (224,224,3)
Processing image 322/1000: 2.5R_8.5_4.png


/tmp/ipykernel_68234/3164171630.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
